In [ ]:
import numpy as np
import json
import os
import matplotlib.pyplot as plt
from IPython.display import display, HTML

Function definitions

In [ ]:
# Define the GaN loss analysis function
def analyze_gan_losses(params=None):
    """
    Analyzes switching and conduction losses in GaN transistors based on the analytical model.

    Parameters:
    -----------
    params : dict, optional
        Dictionary of parameters to override default values

    Returns:
    --------
    dict
        Comprehensive results including losses, timings, efficiency metrics
    """
    # Use provided parameters or use defaults
    if params is None:
        params = {}

    # Buck converter operating conditions
    Vin = params.get('Vin', 400)  # Input voltage (V)
    Vout = params.get('Vout', 48)  # Output voltage (V)
    Iout = params.get('Iout', 10)  # Output current (A)
    D = Vout / Vin  # Duty cycle

    # Constants and parameters
    Vds = params.get('Vds', Vin)  # Drain-to-source voltage (V)
    Ipeak = Iout / D  # Peak current through HS switch (A)
    Ids = params.get('Ids', Ipeak / 2)  # Average current during switching
    Rgon = params.get('Rgon', 0.2)  # Gate resistance during turn-on (ohms)
    Rgoff = params.get('Rgoff', 0.1)  # Gate resistance during turn-off (ohms)
    Vdr_on = params.get('Vdr_on', 7)  # Gate drive voltage during turn-on (V)
    Vdr_off = params.get('Vdr_off', -3)  # Gate drive voltage during turn-off (V)
    Vth = params.get('Vth', 1.3)  # Threshold voltage (V)
    Vgsmiller = params.get('Vgsmiller', 3.0)  # Miller plateau voltage (V)
    gfs = params.get('gfs', 100)  # Transconductance (S), typical estimated value
    fsw = params.get('fsw', 100e3)  # Switching frequency (Hz)
    Ls = params.get('Ls', 0.5e-9)  # Common source inductance (H)
    Rds_on = params.get('Rds_on', 50e-3)  # On-state resistance (ohms)

    # Capacitances (from datasheet in farads)
    Ciss_hv = params.get('Ciss_hv', 235e-12)
    Coss_hv = params.get('Coss_hv', 60e-12)
    Crss_hv = params.get('Crss_hv', 0.6e-12)
    Ciss_lv = params.get('Ciss_lv', 235e-12)
    Coss_lv = params.get('Coss_lv', 60e-12)
    Crss_lv = params.get('Crss_lv', 80e-12)

    # Derived capacitances at high and low voltages
    Cgshv, Cgdhv = Ciss_hv - Crss_hv, Crss_hv
    Cgslv, Cgdlv = Ciss_lv - Crss_lv, Crss_lv

    # Helper functions for dynamic values
    Crss_dynamic = lambda Vds: 5.679e-10 * (Vds ** -0.7523)  # Equation (12)

    def turn_on_loss():
        # Stage t0 to t1
        ton1_t0 = -Rgon * (Cgdhv + Cgshv) * np.log(1 - (Vth - Vdr_off) / (Vdr_on - Vdr_off))

        # Stage t1 to t2
        ton2_t1 = (Ids * (Rgon * (Cgdhv + Cgshv) + Ls * gfs)) / ((Vdr_on - Vgsmiller) * gfs)

        # Stage t2 to tP
        depth = 0.9
        Qsto = (Coss_lv + Crss_lv) * (1 - depth) * Vds  # Adjusted dynamic capacitance
        di_dt = Ids / ton2_t1
        Isto = np.sqrt(max(Qsto * di_dt, 0))  # Ensure non-negative value
        Vgsmiller_peak = Isto / gfs + Vgsmiller
        tonP_t1 = (Ids * (Rgon * (Cgdhv + Cgshv) + Ls * gfs)) / ((Vdr_on - Vgsmiller_peak) * gfs)

        # Stage tP to tD
        Igdon = (Vdr_on - Vgsmiller) / Rgon
        Crss_prime = Crss_dynamic(Vds)
        tonD_tP = Crss_prime * Vds / Igdon

        # Stage tP to t3
        ton3_tP = Cgdlv * (1 - depth) * Vds * Rgon / (Vdr_on - Vgsmiller)

        # Stage t3 to t4
        ton4_t3 = -Rgon * (Cgdlv + Cgslv) * np.log(1 - depth * (Vdr_on - Vgsmiller) / (Vdr_on - Vdr_off))

        # Total turn-on time
        total_turn_on_time = ton1_t0 + ton2_t1 + tonP_t1 + tonD_tP + ton3_tP + ton4_t3

        # Total turn-on loss
        Pon = (
            Vds * (Ids + Isto) * (tonP_t1 / 2) +
            Vds * Isto * (tonD_tP / 3) +
            Vds * Ids * (tonD_tP / 2)
        ) * fsw

        # Return detailed time components for analysis/visualization
        time_components = {
            'ton1_t0': ton1_t0,
            'ton2_t1': ton2_t1,
            'tonP_t1': tonP_t1,
            'tonD_tP': tonD_tP,
            'ton3_tP': ton3_tP,
            'ton4_t3': ton4_t3
        }

        return max(Pon, 0), total_turn_on_time, Isto, time_components

    def turn_off_loss():
        depth = 0.9
        # Stage t0 to t1
        toff1_t0 = -Rgoff * (Cgdlv + Cgslv) * np.log((Vdr_on - Vgsmiller) / (Vdr_on - Vdr_off))

        # Stage t1 to t2
        toff2_t1 = (Cgdlv * Vds * Rgoff) / ((0.5 * (Vgsmiller + Vth) - Vdr_off))

        # Stage t2 to t3
        toff3_t2 = -Rgoff * (Cgdhv + Cgshv) * np.log((Vth - Vdr_off) / (depth * Vth - Vdr_off))

        # Total turn-off time
        total_turn_off_time = toff1_t0 + toff2_t1 + toff3_t2

        # Total turn-off loss
        Poff = (Vds * Ids / 6) * max(toff2_t1, 0) * fsw

        # Return detailed time components for analysis/visualization
        time_components = {
            'toff1_t0': toff1_t0,
            'toff2_t1': toff2_t1,
            'toff3_t2': toff3_t2
        }

        return max(Poff, 0), total_turn_off_time, time_components

    def conduction_loss():
        # Conduction loss calculation
        Pcond = Iout**2 * Rds_on * D
        return Pcond

    # Calculate losses
    Pon, ton_total, Isto_peak, ton_components = turn_on_loss()
    Poff, toff_total, toff_components = turn_off_loss()
    Pcond = conduction_loss()

    # Total switching and conduction loss
    Ptotal = Pon + Poff + Pcond

    # Calculate output power and efficiency
    output_power = Vout * Iout
    input_power = output_power + Ptotal
    efficiency = 100 * output_power / input_power if input_power > 0 else 0

    # Create results dictionary
    results = {
        "operating_conditions": {
            "Vin": float(Vin),
            "Vout": float(Vout),
            "Iout": float(Iout),
            "Duty_cycle": float(D),
            "Switching_frequency": float(fsw)
        },
        "losses": {
            "turn_on_loss_W": float(Pon),
            "turn_off_loss_W": float(Poff),
            "conduction_loss_W": float(Pcond),
            "total_loss_W": float(Ptotal)
        },
        "timings": {
            "turn_on_time_s": float(ton_total),
            "turn_off_time_s": float(toff_total),
            "turn_on_components": {k: float(v) for k, v in ton_components.items()},
            "turn_off_components": {k: float(v) for k, v in toff_components.items()}
        },
        "current": {
            "peak_current_A": float(Ipeak),
            "current_overshoot_A": float(Isto_peak)
        },
        "efficiency": {
            "output_power_W": float(output_power),
            "input_power_W": float(input_power),
            "efficiency_percent": float(efficiency)
        },
        "device_parameters": {
            "Rds_on": float(Rds_on),
            "gate_resistances": {"Rgon": float(Rgon), "Rgoff": float(Rgoff)},
            "capacitances": {
                "high_voltage": {"Ciss": float(Ciss_hv), "Coss": float(Coss_hv), "Crss": float(Crss_hv)},
                "low_voltage": {"Ciss": float(Ciss_lv), "Coss": float(Coss_lv), "Crss": float(Crss_lv)}
            }
        }
    }

    return results




In [ ]:
# Run analysis with default parameters
default_results = analyze_gan_losses()

# Display formatted results
def display_results(results):
    """Display formatted results in a notebook-friendly way"""
    html = f"""
    <h3>GaN Loss Analysis Results</h3>
    <table style="width:80%; border-collapse: collapse;">
        <tr style="background-color: #f2f2f2;">
            <th style="padding: 8px; text-align: left; border: 1px solid #ddd;">Loss Type</th>
            <th style="padding: 8px; text-align: right; border: 1px solid #ddd;">Value (W)</th>
            <th style="padding: 8px; text-align: right; border: 1px solid #ddd;">Percentage</th>
        </tr>
        <tr>
            <td style="padding: 8px; border: 1px solid #ddd;">Turn-on Loss</td>
            <td style="padding: 8px; text-align: right; border: 1px solid #ddd;">{results['losses']['turn_on_loss_W']:.3f}</td>
            <td style="padding: 8px; text-align: right; border: 1px solid #ddd;">
                {100 * results['losses']['turn_on_loss_W'] / results['losses']['total_loss_W']:.1f}%
            </td>
        </tr>
        <tr>
            <td style="padding: 8px; border: 1px solid #ddd;">Turn-off Loss</td>
            <td style="padding: 8px; text-align: right; border: 1px solid #ddd;">{results['losses']['turn_off_loss_W']:.3f}</td>
            <td style="padding: 8px; text-align: right; border: 1px solid #ddd;">
                {100 * results['losses']['turn_off_loss_W'] / results['losses']['total_loss_W']:.1f}%
            </td>
        </tr>
        <tr>
            <td style="padding: 8px; border: 1px solid #ddd;">Conduction Loss</td>
            <td style="padding: 8px; text-align: right; border: 1px solid #ddd;">{results['losses']['conduction_loss_W']:.3f}</td>
            <td style="padding: 8px; text-align: right; border: 1px solid #ddd;">
                {100 * results['losses']['conduction_loss_W'] / results['losses']['total_loss_W']:.1f}%
            </td>
        </tr>
        <tr style="font-weight: bold; background-color: #f9f9f9;">
            <td style="padding: 8px; border: 1px solid #ddd;">Total Loss</td>
            <td style="padding: 8px; text-align: right; border: 1px solid #ddd;">{results['losses']['total_loss_W']:.3f}</td>
            <td style="padding: 8px; text-align: right; border: 1px solid #ddd;">100.0%</td>
        </tr>
    </table>
    <br>
    <p><b>Efficiency:</b> {results['efficiency']['efficiency_percent']:.2f}%</p>
    """
    display(HTML(html))

# Function to plot loss breakdown
def plot_loss_breakdown(results):
    """Create pie chart of loss breakdown"""
    labels = ['Turn-On Loss', 'Turn-Off Loss', 'Conduction Loss']
    sizes = [
        results['losses']['turn_on_loss_W'],
        results['losses']['turn_off_loss_W'],
        results['losses']['conduction_loss_W']
    ]
    colors = ['#ff9999','#66b3ff','#99ff99']
    explode = (0.1, 0, 0)  # explode the 1st slice (Turn-On Loss)

    plt.figure(figsize=(8, 6))
    plt.pie(sizes, explode=explode, labels=labels, colors=colors,
            autopct='%1.1f%%', shadow=True, startangle=90)
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
    fsw_khz = results['operating_conditions']['Switching_frequency'] / 1000
    plt.title(f'GaN Loss Distribution at {fsw_khz:.0f} kHz')
    plt.tight_layout()
    plt.show()

# Function to export results to JSON
def export_to_json(results, filename="gan_losses.json"):
    """Export results to a JSON file"""
    with open(filename, 'w') as f:
        json.dump(results, f, indent=4)
    print(f"Results exported to {os.path.abspath(filename)}")
    return os.path.abspath(filename)

# Function for parameter sweeps
def frequency_sweep(freq_range, base_params=None):
    """
    Perform a frequency sweep and analyze how losses change

    Parameters:
    -----------
    freq_range : list or array
        List of frequencies to analyze
    base_params : dict, optional
        Base parameters to use for the sweep

    Returns:
    --------
    dict
        Results of the sweep for plotting
    """
    if base_params is None:
        base_params = {}

    results = {
        'frequencies': [],
        'turn_on_losses': [],
        'turn_off_losses': [],
        'conduction_losses': [],
        'total_losses': [],
        'efficiencies': []
    }

    for freq in freq_range:
        params = base_params.copy()
        params['fsw'] = freq

        analysis = analyze_gan_losses(params)

        results['frequencies'].append(freq/1000)  # kHz
        results['turn_on_losses'].append(analysis['losses']['turn_on_loss_W'])
        results['turn_off_losses'].append(analysis['losses']['turn_off_loss_W'])
        results['conduction_losses'].append(analysis['losses']['conduction_loss_W'])
        results['total_losses'].append(analysis['losses']['total_loss_W'])
        results['efficiencies'].append(analysis['efficiency']['efficiency_percent'])

    return results

# Function to plot frequency sweep results
def plot_frequency_sweep(sweep_results):
    """
    Plot the results of a frequency sweep

    Parameters:
    -----------
    sweep_results : dict
        Results from frequency_sweep function
    """
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

    # Plot losses
    ax1.plot(sweep_results['frequencies'], sweep_results['turn_on_losses'], 'r-', label='Turn-On Loss')
    ax1.plot(sweep_results['frequencies'], sweep_results['turn_off_losses'], 'b-', label='Turn-Off Loss')
    ax1.plot(sweep_results['frequencies'], sweep_results['conduction_losses'], 'g-', label='Conduction Loss')
    ax1.plot(sweep_results['frequencies'], sweep_results['total_losses'], 'k-', linewidth=2, label='Total Loss')

    ax1.set_ylabel('Power Loss (W)')
    ax1.set_title('GaN Transistor Losses vs. Switching Frequency')
    ax1.legend()
    ax1.grid(True)

    # Plot efficiency
    ax2.plot(sweep_results['frequencies'], sweep_results['efficiencies'], 'b-', linewidth=2)
    ax2.set_xlabel('Frequency (kHz)')
    ax2.set_ylabel('Efficiency (%)')
    ax2.set_title('Efficiency vs. Switching Frequency')
    ax2.grid(True)

    # Add a small margin to y-axis limits for better visibility
    y_min = min(sweep_results['efficiencies']) - 0.1
    y_max = 100
    ax2.set_ylim(max(y_min, 90), y_max)  # Limit to minimum 90% for better visibility

    plt.tight_layout()
    plt.show()

Example Usage

In [ ]:
display_results(default_results)
plot_loss_breakdown(default_results)

# Example - Export to JSON
# json_file = export_to_json(default_results)

# Example - Run a frequency sweep from 50kHz to 500kHz
# freq_range = np.linspace(50e3, 500e3, 20)
# sweep_results = frequency_sweep(freq_range)
# plot_frequency_sweep(sweep_results)

# Example - Custom parameter analysis
custom_params = {
    'Vin': 400,
    'Vout': 48,
    'Iout': 15,  # Higher current
    'fsw': 200e3,  # Higher frequency
    'Rgon': 0.5,  # Higher gate resistance
    'Rgoff': 0.2
}

# Uncomment to run custom analysis
# custom_results = analyze_gan_losses(custom_params)
# display_results(custom_results)
# plot_loss_breakdown(custom_results)
